In [5]:
#written by Stefan Solagratio Simanjuntak
#1039092


import requests
import html

from bs4 import BeautifulSoup

from re import sub
from decimal import Decimal

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def removeall_l_f(l,element):
    
    try:
        while True:
            l.remove(element)
    except ValueError:
        pass 
    
    return l

def nan_empty_val(d):
    
    for key in d:
        
        if d[key] == []:
            d[key].append(float("nan"))
            
    return d;

def process_land(data_s):
    
    meter_bool =False;
    
    val = False;
    
        
    data_l = data_s.split('\xa0');
    
    if len(data_l)==1:
        return data_s;
        
    n = 0;
    
        
    for item in data_l:
        if 'm' in item:
            
            if data_l[n-1] != 'sq':
                val = float(data_l[n-1]);
            else:
                val = float(data_l[n-2])
            meter_bool == True;
        n+=1;
                
    n = 0;
            
    if meter_bool == False:
        for item in data_l:
            if 'f' in item:

                if data_l[n-1] != 'sq':
                    val = float(data_l[n-1])/10.764;
                else:
                    val = float(data_l[n-2])/10.764
            
            n+=1;
            
    if val == False:
        
        return float("nan")
            
    return val

def get_attribute(prop_attribute_list,newclass_l,newtype_l):
    
    data_d = {'bed':[],'bath':[],'car':[],'land':[],'type':[],'address':[]};
    newclass_l = [];
    newtype_l = [];

    for i in range(0,len(prop_attribute_list)):
        
        if prop_attribute_list[i].get('class')[0] == "address":
            data_d['address'] = prop_attribute_list[i].text;
            continue;
    
        if (prop_attribute_list[i].get('class')[0] != "property-meta"):
            class_s = prop_attribute_list[i].get('class')[0];
        else:
            class_s = prop_attribute_list[i].get('class')[1];
            
        
        data_s = prop_attribute_list[i].find(text=True, recursive=False);
        
        
        if class_s == 'type':
            
            data_s = data_s.replace(' ','')
            
            if data_s not in newtype_l:
                newtype_l.append(data_s);
            
            data_d[class_s].append(data_s);
            continue;
    
        data_l = data_s.split(' ');
        data_l = removeall_l_f(data_l,'')
    
        land_bool = False;
    
        if class_s=='land':
            land_bool = True;
            val = process_land(data_s);
                
        try:
            if land_bool == False:
                val = float(data_l[0]);
        except:
            val = data_s;
    
        try:
            data_d[class_s].append(val);
        except:
            newclass_l.append(val);
            
    data_d = nan_empty_val(data_d);
            
    return data_d,newclass_l,newtype_l

def get_historical_price(prop_hstp_list):
    data_d = {'month':[],'year':[],'rent':[]};

    weeks_in_year_i = 52;
    weeks_in_month_i = 4.34524;
    
    na_i = 0;
    nm_i = 0;
    nw_i = 0;
    
    first_bool = False;
    

    for i in range(0,len(prop_hstp_list)):
        
        date_s = prop_hstp_list[i].find('span').text
        data_s = prop_hstp_list[i].find(text=True, recursive=False);
        [month_s,year_s] = date_s.split(' ');
        
        try:
            if has_numbers(data_s) == False:
                continue;
        except:
            continue;
            
        
        
        data_l = data_s.split(' ')
    
        for info_s in data_l:
    
            if '$' in info_s:
                value = float(Decimal(sub(r'[^\d.]', '', info_s)))
    
        data_d['month'].append(month_s);
        data_d['year'].append(year_s);
    
        if 'a' in data_s:
        
            data_d['rent'].append(value/weeks_in_year_i);
            na_i+=1;
        
        elif 'm' in data_s:
        
            data_d['rent'].append(value/weeks_in_month_i);
            nm_i+=1;
        
        elif 'w' in data_s:
        
            data_d['rent'].append(value);
            nw_i+=1;
        
        else:
        
            if (nw_i>na_i) and (nw_i>nm_i):
                data_d['rent'].append(value);
            
            if (nm_i>nw_i) and (nm_i>na_i):
                data_d['rent'].append(value/weeks_in_month_i);
            
            if (na_i>nw_i) and (na_i>nm_i):
                data_d['rent'].append(value/weeks_in_year_i);   
            else:
                data_d['rent'].append(value);
                first_bool = True;
    
    if first_bool == True:
            
        if (nm_i>nw_i) and (nm_i>na_i):
            data_d['rent'][0]=data_d['rent'][0]/weeks_in_month_i;
            
        if (na_i>nw_i) and (na_i>nm_i):
            data_d['rent'][0]=data_d['rent'][0]/weeks_in_year_i; 
    
    return data_d
            
    



    

In [ ]:
# max 783 pages    
n_pages_i = 4;

newclass_l = [];
newtype_l = [];
columns = ['year','month','bed','bath','car','land','type','address', 'rent'];
data_l = [];


headers = { 'User-Agent':\
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
           'Accept-Language':'en-GB,en-US;q=0.9,en;q=0.8',
           'Referer':'https://google.com',
           'DNT':'1'
          }

#https://www.oldlistings.com.au/real-estate/VIC/Abbotsford/3067/rent/

In [6]:
for pg_i in range(1,n_pages_i):
    
    if pg_i == 1:
        pg_str = '';
    else:
        pg_str = str(pg_i);

    page = requests.\
    get('https://www.oldlistings.com.au/real-estate/VIC/Melbourne/3000/rent/'+pg_str,headers = headers);
    soup = BeautifulSoup(page.content, 'html.parser');
    prop_div= soup.find_all("div", {"class":\
                                ["property odd clearfix","property even clearfix"]});

    for data_i in range(0,len(prop_div)):
        
        attribute_d = [];
        price_d = [];
        prop_html = list(prop_div[data_i].children)[1];
        prop_list = list(prop_html.children);
        prop_list = removeall_l_f(prop_list,' ');
        prop_attribute_html = prop_list[0];
        prop_attribute_list = list(prop_attribute_html);
        

        prop_attribute_list = removeall_l_f(prop_attribute_list,' ');
        attribute_d, newattribute_l,newtype_l =  get_attribute(prop_attribute_list,newclass_l,newtype_l)
        
        

        prop_hstp_html = prop_list[2];
        prop_hstp_list = list(list(prop_hstp_html)[3]);
        prop_hstp_list = removeall_l_f(prop_hstp_list,' ');
        
        
        
        price_d = get_historical_price(prop_hstp_list);
        
        
    
        n_row = len(price_d['year']);
    
    
        for i in range(n_row):
    
            data_l.append(\
                  
                          ( price_d['year'][i],\
                          price_d['month'][i],\
                          attribute_d['bed'][0],\
                          attribute_d['bath'][0],\
                          attribute_d['car'][0],\
                          attribute_d['land'][0],\
                          attribute_d['type'][0],\
                          attribute_d['address'],\
                          price_d['rent'][i]
                          )\
                     
                         )
        

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.createDataFrame(data_l,columns);
df.write.parquet("../data/raw/melbourne_past_listings.parquet");


In [33]:
import csv

with open('../data/raw/melbourne_past_listings.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(columns)
    for row in data_l:
        csv_out.writerow(row)
        
with open('melbourne_past_listings.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(columns)
    for row in data_l:
        csv_out.writerow(row)


In [35]:
import pandas

past_df = pandas.read_csv('../data/raw/melbourne_past_listings.csv')

/Users/stefan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
